In [1]:
import os, sys

repo_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
src_dir = os.path.join(repo_root, "src")
if src_dir not in sys.path:
    sys.path.append(src_dir)

src_dir

'/Users/adithsrinivasan/Documents/GitHub/applied-portfolio-optimization/src'

In [2]:
import sys
print(sys.executable)

/Users/adithsrinivasan/Documents/GitHub/applied-portfolio-optimization/.venv/bin/python


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import moment, kstat, skew, kurtosis
import yfinance as yf
import riskfolio as rp
import warnings

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

In [4]:
# Date range
start = '2019-01-01'
end = '2023-12-30'

In [5]:
# Tickers for assets
assets = ['APA', 'BA', 'BAX', 'BMY', 'CMCSA',
          'CNP', 'CPB', 'DE', 'HPQ', 'JCI']
assets.sort()

In [6]:
prices = yf.download(assets, start=start, end=end, auto_adjust=False)
prices = prices.loc[:,('Adj Close', slice(None))]
prices.columns = assets

prices

[*********************100%***********************]  10 of 10 completed


,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI
Date,,,,,,,,,,
2019-01-02,2254.7834%,31464.5142%,5741.0908%,4047.3595%,2880.5264%,2285.5669%,2548.6446%,13438.8260%,1622.7064%,2626.1440%
2019-01-03,2260.6050%,30210.0555%,5668.9983%,3510.5118%,2903.1549%,2298.6269%,2580.2454%,13071.3242%,1567.6463%,2583.9535%
2019-01-04,2355.4207%,31782.2571%,5846.5946%,3648.2246%,3001.2123%,2341.0738%,2594.4670%,13763.6810%,1629.7855%,2687.2776%
2019-01-07,2406.1554%,31882.3456%,5817.5812%,3766.4867%,2969.3636%,2350.0528%,2661.6194%,13988.7146%,1647.8769%,2731.1893%
2019-01-08,2441.9195%,33089.1876%,5801.7559%,3733.8093%,2993.6695%,2394.9482%,2681.5777%,14239.1663%,1668.3279%,2750.1324%
...,...,...,...,...,...,...,...,...,...,...
2023-12-22,3301.9859%,26044.0002%,3664.0263%,4741.6355%,4116.1064%,2693.2505%,3983.5262%,38557.3425%,2795.5677%,5512.9955%
2023-12-26,3424.1116%,26279.0009%,3683.1207%,4665.4652%,4109.5585%,2704.6385%,3981.6734%,38989.7308%,2816.8728%,5599.4701%
2023-12-27,3413.0920%,26210.0006%,3690.7574%,4643.7016%,4115.1711%,2693.2505%,3969.6304%,39025.6805%,2801.1257%,5604.3282%


In [7]:
returns = prices[assets].pct_change().dropna()

returns

,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI
Date,,,,,,,,,,
2019-01-03,0.2582%,-3.9869%,-1.2557%,-13.2641%,0.7856%,0.5714%,1.2399%,-2.7346%,-3.3931%,-1.6066%
2019-01-04,4.1943%,5.2042%,3.1328%,3.9229%,3.3776%,1.8466%,0.5512%,5.2968%,3.9639%,3.9987%
2019-01-07,2.1540%,0.3149%,-0.4962%,3.2416%,-1.0612%,0.3835%,2.5883%,1.6350%,1.1100%,1.6341%
2019-01-08,1.4864%,3.7853%,-0.2720%,-0.8676%,0.8186%,1.9104%,0.7499%,1.7904%,1.2410%,0.6936%
2019-01-09,5.1090%,0.9691%,1.7124%,-1.6878%,0.9518%,-1.2270%,1.7565%,-1.6505%,0.0000%,0.0939%
...,...,...,...,...,...,...,...,...,...,...
2023-12-22,0.0557%,-0.6030%,-1.3621%,2.0094%,-0.0454%,0.2827%,0.5377%,0.8847%,0.1327%,1.3757%
2023-12-26,3.6986%,0.9023%,0.5211%,-1.6064%,-0.1591%,0.4228%,-0.0465%,1.1214%,0.7621%,1.5686%
2023-12-27,-0.3218%,-0.2626%,0.2073%,-0.4665%,0.1366%,-0.4211%,-0.3025%,0.0922%,-0.5590%,0.0868%


In [8]:
mu = returns.mean()

mu

APA      0.1386%
BA       0.0355%
BAX     -0.0187%
BMY      0.0217%
CMCSA    0.0436%
CNP      0.0345%
CPB      0.0484%
DE       0.1072%
HPQ      0.0725%
JCI      0.0776%
dtype: float64

In [9]:
# Population Central Moments
sigma_2_p = moment(returns, moment=2, axis=0)
sigma_3_p = moment(returns, moment=3, axis=0)
sigma_4_p = moment(returns, moment=4, axis=0)

# Sample Central Moments
sigma_2_s = kstat(returns, n=2, axis=0)
sigma_3_s = kstat(returns, n=3, axis=0)
sigma_4_s = kstat(returns, n=4, axis=0)

# Standardized Central Moments
std_sigma_3 = skew(returns, axis=0)
std_sigma_4 = kurtosis(returns, fisher=True, axis=0)

moments = pd.DataFrame(
    data = [sigma_2_p,
            sigma_3_p,
            sigma_4_p,
            sigma_2_s,
            sigma_3_s,
            sigma_4_s,
            std_sigma_3,
            std_sigma_4],
    index = [
        "Population Variance",
        "Population Skewness",
        "Population Kurtosis",
        "Sample Variance",
        "Sample Skewness",
        "Sample Kurtosis",
        "Standardized Skewness",
        "Standardized Kurtosis"
    ],
    columns = assets
)

moments

,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI
Population Variance,0.2027%,0.1008%,0.0323%,0.0210%,0.0309%,0.0411%,0.0250%,0.0445%,0.0582%,0.0344%
Population Skewness,-0.0096%,0.0009%,-0.0006%,-0.0002%,0.0000%,-0.0006%,0.0001%,-0.0003%,-0.0007%,-0.0004%
Population Kurtosis,0.0106%,0.0016%,0.0001%,0.0001%,0.0001%,0.0005%,0.0001%,0.0002%,0.0004%,0.0001%
Sample Variance,0.2029%,0.1009%,0.0323%,0.0210%,0.0309%,0.0411%,0.0250%,0.0446%,0.0583%,0.0344%
Sample Skewness,-0.0096%,0.0009%,-0.0006%,-0.0003%,0.0000%,-0.0006%,0.0001%,-0.0003%,-0.0007%,-0.0004%
Sample Kurtosis,0.0095%,0.0013%,0.0001%,0.0000%,0.0001%,0.0004%,0.0001%,0.0001%,0.0003%,0.0001%
Standardized Skewness,-105.3431%,26.8180%,-102.3818%,-82.0396%,3.2253%,-75.1994%,14.9580%,-32.5262%,-47.3421%,-64.8793%
Standardized Kurtosis,2287.1383%,1227.1337%,1018.3006%,878.9051%,636.0741%,2372.7022%,824.4304%,618.2814%,966.7999%,516.7620%
